In [1]:
import numpy as np
import pickle
import shutil
import os
import pandas as pd
from tempfile import TemporaryDirectory
from numpy.testing import assert_equal, assert_almost_equal, assert_raises

In [201]:
class MovieDB:
    def __init__(self, path):
        self.data_dir = path
        os.makedirs(self.data_dir, exist_ok=True)
        
        return
    
    def add_movie(self, title, year, genre, director):
        if os.path.isfile(os.path.join(self.data_dir,'movies.csv')):
            self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
         #os.path.join(s)   
        else:
            self.movies = pd.DataFrame(columns=['movie_id', 'title', 'year', 'genre', 'director_id'])
            self.movies.to_csv(os.path.join(self.data_dir,'movies.csv'), index=False)
            self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
            
        if os.path.isfile(os.path.join(self.data_dir,'directors.csv')):
            self.df_dir = pd.read_csv(os.path.join(self.data_dir,'directors.csv'))
            
        else:
            self.directors = pd.DataFrame(columns=['director_id', 'given_name', 'last_name'])
            self.directors.to_csv(os.path.join(self.data_dir,'directors.csv'), index=False) 
            self.df_dir = pd.read_csv(os.path.join(self.data_dir,'directors.csv'))
            
        ###------------directors.csv appending-------------------   
        last_name, given_name = director.split(',')
        given_name = given_name.strip()
        last_name = last_name.strip()
        
        name1_bool = self.df_dir['given_name'].str.lower().isin([given_name.lower()])
        name2_bool = self.df_dir['last_name'].str.lower().isin([last_name.lower()])
        
        if len(self.df_dir[(name1_bool)&(name2_bool)]) == 1:
            director_id = self.df_dir[(name1_bool)&(name2_bool)]['director_id'].item()
        else:
            new_id = len(self.df_dir) + 1

            add_dir = {'director_id': new_id, 
                      'given_name': given_name,
                      'last_name': last_name}
            df_add = pd.DataFrame(add_dir, index=[0])
            df_add.to_csv(os.path.join(self.data_dir,'directors.csv'), mode='a', header=False, index=False)
            director_id = new_id
            
            self.df_dir = pd.read_csv(os.path.join(self.data_dir,'directors.csv'))
        ##------------movies.csv appending-------------------
        title_bool = self.df_movies['title'].str.lower().isin([title.lower().strip()])
        year_bool = self.df_movies['year'].isin([year])
        genre_bool = self.df_movies['genre'].str.lower().isin([genre.lower().strip()])
        director_id_bool = self.df_movies['director_id'].isin([director_id])
        
        if len(self.df_movies[(title_bool)&(year_bool)&(genre_bool)&(director_id_bool)] == 1):
            raise MovieDBError
        else:
            movie_id = len(self.df_movies) + 1

            add_mov = {'movie_id': movie_id,
                      'title': title,
                      'year': year, 
                      'genre': genre,
                      'director_id': director_id}
            df_addmov = pd.DataFrame(add_mov, index=[0])
            df_addmov.to_csv(os.path.join(self.data_dir,'movies.csv'), mode='a', header=False, index=False)
           
        self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
        
        return movie_id
    
    def add_movies(self,movies):
        # check if movies.csv exists already, if not create a new file
        print(self.data_dir)
        if os.path.isfile(os.path.join(self.data_dir,'movies.csv')):
            self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
            print('movies.csv exists')
        else:
            self.movies = pd.DataFrame(columns=['movie_id', 'title', 'year', 'genre', 'director_id'])
            self.movies.to_csv(os.path.join(self.data_dir,'movies.csv'), index=False)
            self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
            print('movies.csv does NOT exist')
        
        # check if directors.csv exists already, if not create a new file
        if os.path.isfile(os.path.join(self.data_dir,'directors.csv')):
            self.df_dir = pd.read_csv(os.path.join(self.data_dir,'directors.csv'))
            print('directors.csv exists')
        else:
            self.directors = pd.DataFrame(columns=['director_id', 'given_name', 'last_name'])
            self.directors.to_csv(os.path.join(self.data_dir,'directors.csv'), index=False) 
            self.df_dir = pd.read_csv(os.path.join(self.data_dir,'directors.csv'))
            print('directors.csv does NOT exist')
            
        # create temporary variables for this function
        
        counter = 0
        new_movie_ids = []
        
        display(self.df_dir)
        # go through all the dictionary of movies in the list
        for movie in movies:
            # keep on repopulating DF for any new movies or directors added
            df_movies_dir = self.df_movies.merge(self.df_dir, how='left', on='director_id')
            
            # check if the dictionary has all the required keys
            if 'title' in movie and 'year' in movie and 'genre' in movie and 'director' in movie:
                # create temporary variables when checking /creating a movie in the list
                # use strip() to clean spaces
                last_name, given_name = movie['director'].split(',')
                given_name = given_name.strip()
                last_name = last_name.strip()
                movie['title'] = movie['title'].strip()
                movie['genre'] = movie['genre'].strip()
                
                
                # check if the movie already in movies.csv and directors.csv
                if movie['title'].lower() in df_movies_dir['title'].str.lower().to_list() and \
                        movie['year'] in df_movies_dir['year'].to_list() and \
                        movie['genre'].lower() in df_movies_dir['genre'].str.lower().to_list() and \
                        given_name.lower() in df_movies_dir['given_name'].str.lower().to_list() and \
                        last_name.lower() in df_movies_dir['last_name'].str.lower().to_list():
                    print('Warning: movie {',movie['title'],'} is already in the database. Skipping...')
                    
                else:
                    ###------------directors.csv appending-------------------   
                    name1_bool = self.df_dir['given_name'].str.lower().isin([given_name.lower()])
                    name2_bool = self.df_dir['last_name'].str.lower().isin([last_name.lower()])

                    if len(self.df_dir[(name1_bool)&(name2_bool)]) == 1:
                        director_id = self.df_dir[(name1_bool)&(name2_bool)]['director_id'].item()
                        
                    else:
                        new_id = len(pd.read_csv(os.path.join(self.data_dir,'directors.csv'))) + 1
                        add_dir = {'director_id': new_id, 
                                  'given_name': given_name,
                                  'last_name': last_name}
                        df_add = pd.DataFrame(add_dir, index=[0])
                        df_add.to_csv(os.path.join(self.data_dir,'directors.csv'), mode='a', header=False, index=False)
                        self.df_dir = pd.read_csv(os.path.join(self.data_dir,'directors.csv'))

                        director_id = new_id
                    
                    ##------------movies.csv appending-------------------
                    movie_id = len(pd.read_csv(os.path.join(self.data_dir,'movies.csv'))) + 1
                    add_mov = {'movie_id': movie_id,
                              'title': movie['title'],
                              'year': movie['year'], 
                              'genre': movie['genre'],
                              'director_id': director_id}
                    df_addmov = pd.DataFrame(add_mov, index=[0])
                    df_addmov.to_csv(os.path.join(self.data_dir,'movies.csv'), mode='a', header=False, index=False)
                    self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
                    
                    # append to new_movie_ids list
                    new_movie_ids.append(movie_id)
            else:
                print('Warning: movie index {',counter,'} has invalid or incomplete information. Skipping...')
            
            # temp variable just for the index of the list of movies "to be added"
            counter += 1
        
        df_movies_dir = self.df_movies.merge(self.df_dir, how='left', on='director_id')
        display(df_movies_dir)
        
        return new_movie_ids


    def search_movies(self, title = None, year = None, genre = None, director_id = None):
        self.df_movies = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))

        # check if the parameter is not present or has the correct data type
        if (type(title) == str or title == None) and \
            (type(year) == int or year == None) and \
            (type(genre) == str or genre == None) and \
            (type(director_id) == int or director_id == None):
            
            # handle none parameters
            temp_title = title
            temp_genre = genre
            if title != None:
                temp_title = temp_title.strip().lower()
            if temp_genre != None:
                temp_genre = temp_genre.strip().lower()
                
            df_temp = self.df_movies[ 
                        (self.df_movies['title'].str.lower() == temp_title) |
                        (self.df_movies['year'] == year) | 
                        (self.df_movies['genre'].str.lower() == temp_genre) | 
                        (self.df_movies['director_id'] == director_id)
                        ]
            
            # convert the column of movie_id to a list
            temp_list = df_temp['movie_id'].tolist()
            
            # check if at least one of the parameters is present
            if title == None and year == None and genre == None and director_id == None:
                raise MovieDBError
            else:
                return temp_list
        def delete_movie(self, movie_id):
            df_to_del = pd.read_csv(os.path.join(self.data_dir,'movies.csv'))
            
            if movie_id not in df_to_del['movie_id'].values:
                raise MovieDBError
            else:
                index = df_to_del[df_to_del['movie_id']==movie_id].index.item()
                df_done = df_to_del.drop(index)
                return 
            
            
class MovieDBError(Exception):
    pass 

In [202]:
path = '.'
movie_db = MovieDB(path)

In [74]:
movie_db.add_movie('Shrek', 2001, 'Comedy', 'Adamson, Andrew')
movie_db.add_movie('Shrek', 2001, 'Comedy', 'Adamson, Andrez')

movies.csv does NOT exist
directors.csv does NOT exist
movies.csv exists
directors.csv exists


In [75]:
movie_db.add_movie('Shrek', 2001, 'Comedy', 'Adamson, Andrews')

movies.csv exists
directors.csv exists


In [76]:
display(movie_db.df_movies)
display(movie_db.df_dir)

,movie_id,title,year,genre,director_id
0,1,Shrek,2001,Comedy,1
1,2,Shrek,2001,Comedy,2
2,3,Shrek,2001,Comedy,3


,director_id,given_name,last_name
0,1,Andrew,Adamson
1,2,Andrez,Adamson
2,3,Andrews,Adamson


In [ ]:
# Token frequency
# Create a method token_freq that returns a dictionary with the token as key and
# the number of times that word appeared in all of the titles as value. A token 
# is defined as a case-folded sequence of non-whitespace characters.

In [14]:
df_movtest = pd.read_csv('movies_test2.csv')


1832

In [37]:
def token_freq(): 
    list_mov = df_movtest['title'].str.lower().str.split().tolist()
    li_to_count = [item for sublist in list_mov for item in sublist]

    from collections import Counter

    return Counter(li_to_count)


In [41]:
tf = token_freq()
tf['the']

1874

In [2]:
df_m30 = pd.read_csv('movies_30.csv')
df_m30.head()

,movie_id,title,year,genre,director_id
0,2,Ferris Bueller's Day Off,1986,Comedy,2
1,3,Top Gun,1986,Action,3
2,4,Aliens,1986,Action,4
3,5,Flight of the Navigator,1986,Adventure,5
4,6,Platoon,1986,Drama,6


In [28]:
def delete_movie(movie_id):
    df_to_del = pd.read_csv('movies_30.csv')
    if movie_id not in df_to_del['movie_id'].values:
        return print('error')
    else:
        index = df_to_del[df_to_del['movie_id']==movie_id].index.item()
        df_done = df_to_del.drop(index)
        return df_done

In [197]:
movie = {}
genre = {}
director = {}

def generate_statistics(stat):
    if stat == 'movie':
        return movie
    elif stat == 'genre':
        return genre
    elif stat == 'director':
    return director
    elif stat == 'all':
    return {'movie': movie,
        'genre': genre,
        'director': director}
    else:
        raise MovieDBError

def populate_dict(movie_list, director_list):
    
    director_id_name = {}

    for row in director_list:
    director_id_name[row['director_id']] = row
    
    for row in movie_list:
    if row['year'] not in movie:
        movie[row['year']] = 1
    else:
        movie[row['year']] += 1
        
    if row['genre'] not in genre:
        genre[row['genre']] = {}
        genre[row['genre']][row['year']] = 1
    else:
        genre[row['genre']][row['year']] += 1

    director_obj = director_id_name[row['director_id']]
        
    director_name = "%s, %s"%(director_obj['last_name'],
                                  director_obj['given_name'])
        
    if director_name not in director:
        director[director_name] = {}
        director[director_name][row['year']] = 1
    else:
        director[director_name][row['year']] += 1

1
3
5
7
9


df_mov2 = pd.read_csv('movies_test2.csv')
df_dir2 = pd.read_csv('directors_test2.csv')
df_mov2.merge(df_dir2, on='director_id', how='left')

In [120]:
def generate_statistics(stat):
    df_mov2 = pd.read_csv('movies_test2.csv')
    df_dir2 = pd.read_csv('directors_test2.csv')
    if stat == 'movie':
        return dict(df_mov2.groupby('year')['movie_id'].count())
    elif stat == 'genre':
        
        d = {}

        for a,b in df_mov2.groupby(['genre'])['year']:

            val = dict(df_mov2[df_mov2['genre']== a].groupby('year')['movie_id'].count())
            d[a]= val
        return d
    elif stat =='director':
        df_merged = df_mov2.merge(df_dir2, on='director_id', how='left')

        full_name = df_merged['last_name'] + ', ' + df_merged['given_name']
        df_merged['full_name'] = full_name

        d={}
        for a in df_merged.full_name.unique():
            val = dict(df_merged[df_merged['full_name']==a].groupby('year')['movie_id'].count())
            d[a] = val

        return d
    elif stat == 'all':
        all_d = {}

        for i in ['movie', 'genre', 'director']:
            if i == 'movie':
                all_val = dict(df_mov2.groupby('year')['movie_id'].count())
            elif i == 'genre':
                d_1 = {}
                for a,b in df_mov2.groupby(['genre'])['year']:
                    val = dict(df_mov2[df_mov2['genre']== a].groupby('year')['movie_id'].count())
                    d_1[a]= val
                all_val = d_1
            elif i == 'director':
                df_merged = df_mov2.merge(df_dir2, on='director_id', how='left')

                full_name = df_merged['last_name'] + ', ' + df_merged['given_name']
                df_merged['full_name'] = full_name

                d_2={}
                for a in df_merged.full_name.unique():
                    val = dict(df_merged[df_merged['full_name']==a].groupby('year')['movie_id'].count())
                    d_2[a] = val
                all_val = d_2

            all_d[i] = all_val
        return all_d
        

In [148]:
import matplotlib.pyplot as plt
def plot_statistics(stat):
    if stat == 'movie'
    gr = self.df_movies.groupby('year')['movie_id'].count()


    fig, ax = plt.subplots(1,1,figsize=(8,5))
    return ax.bar(gr.index,gr.values);

    